In [2]:
import pandas as pd
import re
from utils import valid_params, load_properties

## Leitura e tratamento dos dados

In [8]:
df = pd.read_json("assets/olx_location_cep_price.json", orient="records", convert_dates=False)
df = df['caracteristicas'].apply(pd.Series)
df.isna().sum()

categoria                    0
tipo                         0
area_util                 1621
quartos                     38
banheiros                  512
vagas_na_garagem           166
cep                          0
municipio                    0
bairro                       0
logradouro                 198
detalhes_do_imovel        1767
detalhes_do_condominio    2629
condominio                3145
iptu                      4430
dtype: int64

In [14]:
df.isna().sum(), 1- 3145/5079, 1- 4430/5079

(categoria                    0
 tipo                         0
 area_util                 1621
 quartos                     38
 banheiros                  512
 vagas_na_garagem           166
 cep                          0
 municipio                    0
 bairro                       0
 logradouro                 198
 detalhes_do_imovel        1767
 detalhes_do_condominio    2629
 condominio                3145
 iptu                      4430
 dtype: int64,
 0.3807836188226029,
 0.1277810592636346)

In [12]:
len(df)

5079

In [4]:
df_imoveis = load_properties()
df_imoveis.isna().sum()

id                 0
value              0
location           0
price_suggested    0
area               0
rooms              0
bathrooms          0
garages            0
district_name      0
dtype: int64

## Base utilizada

In [3]:
df_imoveis.head(3)

,id,value,location,price_suggested,categoria,area,rooms,bathrooms,garages,district_name
3,6074590677dee513e95c7f1f,200001.0,"{'lat': -10.9299886, 'lng': -37.0830428}",199906.0,Apartamentos,61.0,2.0,2.0,1.0,Jabotiana
6,6074591277dee513e95c7f22,430000.0,"{'lat': -10.9926775, 'lng': -37.0552954}",369970.0,Apartamentos,0.0,2.0,2.0,1.0,Atalaia
11,6074592477dee513e95c7f27,480000.0,"{'lat': -10.9282507, 'lng': -37.056637}",554047.0,Apartamentos,140.0,4.0,4.0,2.0,Salgado Filho


## Bairros válidos

In [4]:
AVAILABLE_DISTRICTS = ['Jabotiana', 'Atalaia', 'Salgado Filho', 'Lamarão', 'Grageru',
                       'Luzia', 'Jardins', 'Farolândia', 'Coroa do Meio', 'Ponto Novo',
                       'Suíssa', 'Centro', 'São Conrado', 'Inácio Barbosa', 'São José',
                       'Zona de Expansão (Mosqueiro)', 'Treze de Julho',
                       'Dezoito do Forte', 'Aeroporto', 'Zona de Expansão (Aruana)',
                       'Siqueira Campos', 'Pereira Lobo', 'Jardim Centenário',
                       'Santa Maria', 'Dom Luciano', 'Olaria', 'Santos Dumont', 'América',
                       'Santo Antônio', 'Getúlio Vargas', 'Cirurgia', 'Industrial',
                       "Porto D'Antas", '18 do Forte', 'José Conrado de Araújo',
                       'Cidade Nova', 'Zona de Expansão (Robalo)', '13 De Julho',
                       'Palestina', 'Soledade', 'Japãozinho']

## Preferências do Usuário 

In [13]:
params = {
    "value": 300000,        # Valor em Reais (R$)
    "rooms": 3,             # Número de quartos
    "area": 80,             # Área do imóvel em metros quadrados
    "bathrooms": 2,         # Número de banheiros do imóvel
    "garages": 1,           # Número de garagens do imóvel
    "district": "Atalaia"   # Bairro onde se localiza o imóvel
}

## Validar parametros 

In [16]:
from utils import valid_params

error = valid_params(params)
if error != None:
    print(error)

## Preparar os parametros

In [20]:
from utils import prepare_params

prepared_params, error = prepare_params(params)

In [21]:
prepared_params

{'value': 300000.0,
 'rooms': 3.0,
 'area': 80.0,
 'bathrooms': 2.0,
 'garages': 1.0,
 'district': 'Atalaia'}

## Alterar a base de acordo com os parametros

In [22]:
from utils import handle_dataframe_values

df_to_algorithm, params_to_algorithm = handle_dataframe_values(df_imoveis, prepared_params)
df_to_algorithm.head(3)

,id,value,location,price_suggested,categoria,area,rooms,bathrooms,garages,district_name,distance,district
3,6074590677dee513e95c7f1f,200001.0,"{'lat': -10.9299886, 'lng': -37.0830428}",199906.0,Apartamentos,61.0,2.0,2.0,1.0,Jabotiana,7.193320,0.0
6,6074591277dee513e95c7f22,430000.0,"{'lat': -10.9926775, 'lng': -37.0552954}",369970.0,Apartamentos,0.0,2.0,2.0,1.0,Atalaia,0.547648,1.0
11,6074592477dee513e95c7f27,480000.0,"{'lat': -10.9282507, 'lng': -37.056637}",554047.0,Apartamentos,140.0,4.0,4.0,2.0,Salgado Filho,6.626499,0.0


## Definição dos pesos de cada caracteristica

In [23]:
weights = {
    'value': [30.0],
    'rooms': [10.0],
    'area': [10.0],
    'bathrooms': [10.0],
    'garages': [10.0],
    'district': [10.0],
    'distance': [10.0]
}

## Aplicando Algoritmo de Recomendação

In [24]:
from utils import mahalanobis_algorithm

sorted_values = mahalanobis_algorithm(df_to_algorithm, params_to_algorithm, weights)

result = {}
for index in range(3):
    aux = df_to_algorithm.iloc[sorted_values.iloc[index].name].to_dict()
    aux['similarity'] = sorted_values.iloc[index][0]
    aux['similarity_index'] = int(sorted_values.iloc[index].name)

    result.update({index: aux})
    df_to_algorithm = df_to_algorithm[df_to_algorithm.id != aux['id']]
    
pd.DataFrame.from_dict(result, orient="index")

,id,value,location,price_suggested,categoria,area,rooms,bathrooms,garages,district_name,distance,district,similarity,similarity_index
0,607f33ac4c5fd7aeae9a73c5,300000.0,"{'lat': -10.9923802, 'lng': -37.0541356}",327663.0,Apartamentos,80.0,3.0,2.0,1.0,Atalaia,0.482151,1.0,2.697004,1413
1,607e8a51092e9f82c3b1eba2,285000.0,"{'lat': -10.9917458, 'lng': -37.0556308}",321778.0,Apartamentos,77.0,3.0,2.0,1.0,Atalaia,0.470860,1.0,3.304200,1325
2,60863bbaed763b6b79f5cb34,170000.0,"{'lat': -10.9720242, 'lng': -37.07319}",154707.0,Apartamentos,65.0,3.0,1.0,1.0,Farolândia,2.793969,0.0,3.377305,2207
